In [24]:
import os
import tensorflow as tf
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from sklearn.preprocessing import StandardScaler

In [29]:

dataCSV = os.path.join(os.getcwd(), "..", "CSGODemo", "Dataset", "CSGODataset.csv")
modelSaveFolder = os.path.join(os.getcwd(), "modelSave")

df = pd.read_csv(dataCSV)

print(df.columns.tolist())

invalidRows = (df == -1).any(axis=1)

print(f"Invalid rows --> {invalidRows}")

print(df)

['Team1_Player1_Kills', 'Team1_Player1_Assists', 'Team1_Player1_Deaths', 'Team1_Player1_HeadshotsPerc', 'Team1_Player1_KR_Ratio', 'Team1_Player1_KD_Ratio', 'Team1_Player1_KAST', 'Team1_Player1_EntryKills', 'Team1_Player1_RWS', 'Team1_Player1_Rating', 'Team1_Player1_Rating2', 'Team1_Player1_ATD', 'Team1_Player1_ADR', 'Team1_Player1_ELO', 'Team1_Player2_Kills', 'Team1_Player2_Assists', 'Team1_Player2_Deaths', 'Team1_Player2_HeadshotsPerc', 'Team1_Player2_KR_Ratio', 'Team1_Player2_KD_Ratio', 'Team1_Player2_KAST', 'Team1_Player2_EntryKills', 'Team1_Player2_RWS', 'Team1_Player2_Rating', 'Team1_Player2_Rating2', 'Team1_Player2_ATD', 'Team1_Player2_ADR', 'Team1_Player2_ELO', 'Team1_Player3_Kills', 'Team1_Player3_Assists', 'Team1_Player3_Deaths', 'Team1_Player3_HeadshotsPerc', 'Team1_Player3_KR_Ratio', 'Team1_Player3_KD_Ratio', 'Team1_Player3_KAST', 'Team1_Player3_EntryKills', 'Team1_Player3_RWS', 'Team1_Player3_Rating', 'Team1_Player3_Rating2', 'Team1_Player3_ATD', 'Team1_Player3_ADR', 'Team1

In [27]:
team_cols = df.columns.str.extract(r'Team(\d+)_(.*?)$')

# Create a new DataFrame to store the extracted information
team_data = pd.DataFrame(team_cols, columns=['Team', 'Stat'])
team_data['Value'] = df.values.flatten()

# Group by team and stat, calculate average
team_avg = team_data.groupby(['Team', 'Stat'])['Value'].mean().unstack()

# Rename columns to match the format you mentioned (e.g., Team1_AverageKills)
team_avg.columns = [f'Team{team}_Average{stat}' for team, stat in team_avg.columns]

# Reset index to get team numbers as columns
team_avg = team_avg.reset_index()

# Display the resulting DataFrame
print(team_avg)

ValueError: Length of values (1362361) does not match length of index (143)

In [13]:
invalidRows = invalidRows.reset_index(drop=True)

cleaneddf = df[df['Team_1_Win?'] != 2]
cleaneddf = cleaneddf.dropna()

print(df.shape)
print(cleaneddf.shape)

(9527, 143)
(9341, 143)


In [21]:
abs_correlations = cleaneddf.corr()[['closeMatch?']].abs()

# Sort absolute correlations in descending order
sorted_abs_correlations = abs_correlations.sort_values(by='closeMatch?', ascending=False)

print(sorted_abs_correlations.head(30))



                             closeMatch?
closeMatch?                     1.000000
ScoreDifference                 0.710349
Team_1_Win?                     0.047331
Team2_Player4_Assists           0.026836
Team2_Player2_ELO               0.026187
Team1_Player2_HeadshotsPerc     0.022317
Team1_Player4_EntryKills        0.021683
Team2_Player3_KAST              0.020332
Team2_Player4_Deaths            0.019618
Team1_Player4_Assists           0.018998
Team1_Player2_ATD               0.018477
Team2_Player4_ELO               0.018029
Team2_Player3_RWS               0.017927
Team2_Player1_Kills             0.017788
Team2_Player1_ELO               0.017356
Team2_Player4_EntryKills        0.017189
Team2_Player3_Rating2           0.016534
Team2_Player1_KD_Ratio          0.016433
Team2_Player2_HeadshotsPerc     0.016373
Team2_Player1_Rating2           0.016127
Team1_Player3_Assists           0.015893
Team2_Player1_KAST              0.015760
Team2_Player1_RWS               0.015694
Team2_Player3_Ra

In [23]:
team1_players = [
    "Team1_Player4_ELO",
    "Team1_Player5_ELO",
    "Team1_Player2_ELO",
    "Team1_Player1_ELO",
    "Team1_Player3_ELO"
]

team2_players = [
    "Team2_Player1_ELO",
    "Team2_Player4_ELO",
    "Team2_Player2_ELO",
    "Team2_Player3_ELO",
    "Team2_Player5_ELO",
]

selected_columns = team1_players + team2_players

ELOFeatures = cleaneddf[selected_columns]
results = cleaneddf['Team_1_Win?']
ScoreDifference = cleaneddf['ScoreDifference']
closeMatch = cleaneddf['closeMatch?']

closeMatch_counts = df['closeMatch?'].value_counts()

print(ELOFeatures.shape)
print(closeMatch.shape)
print(closeMatch_counts)

(9341, 10)
(9341,)
0    6318
1    3209
Name: closeMatch?, dtype: int64


In [16]:
scaler = StandardScaler()

scaled_features = scaler.fit_transform(ELOFeatures)

X_train, X_test, y_train, y_test = train_test_split(scaled_features, closeMatch, test_size=0.2, random_state=42)
score_train, score_test = train_test_split(ScoreDifference, test_size=0.2, random_state=42)

X_train_size = X_train.shape[0]  # Get the number of samples in X_train
X_test_size = X_test.shape[0]    # Get the number of samples in X_test


print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)
print(score_train.shape)
print(score_test.shape)


(7472, 10)
(1869, 10)
(7472,)
(1869,)
(7472,)
(1869,)


In [18]:
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from sklearn.metrics import mean_squared_error
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score, recall_score, f1_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import roc_curve, roc_auc_score
from skopt import BayesSearchCV
from sklearn.model_selection import train_test_split, RepeatedStratifiedKFold

params = dict()
params['n_estimators'] = (10, 500)
params['max_depth'] = (1, 32)

crossVal = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=1)
clf = RandomForestClassifier(verbose=1)

searcher = BayesSearchCV(estimator=clf, search_spaces=params, n_jobs=6, cv=crossVal, n_iter=20, verbose=2)

searcher.fit(X_train, y_train)

print(searcher.best_score_)
print(searcher.best_params_)

y_pred = searcher.predict(X_test)


Fitting 30 folds for each of 1 candidates, totalling 30 fits
Fitting 30 folds for each of 1 candidates, totalling 30 fits
Fitting 30 folds for each of 1 candidates, totalling 30 fits
Fitting 30 folds for each of 1 candidates, totalling 30 fits
Fitting 30 folds for each of 1 candidates, totalling 30 fits
Fitting 30 folds for each of 1 candidates, totalling 30 fits
Fitting 30 folds for each of 1 candidates, totalling 30 fits
Fitting 30 folds for each of 1 candidates, totalling 30 fits
Fitting 30 folds for each of 1 candidates, totalling 30 fits
Fitting 30 folds for each of 1 candidates, totalling 30 fits
Fitting 30 folds for each of 1 candidates, totalling 30 fits
Fitting 30 folds for each of 1 candidates, totalling 30 fits
Fitting 30 folds for each of 1 candidates, totalling 30 fits
Fitting 30 folds for each of 1 candidates, totalling 30 fits
Fitting 30 folds for each of 1 candidates, totalling 30 fits
Fitting 30 folds for each of 1 candidates, totalling 30 fits
Fitting 30 folds for eac

[Parallel(n_jobs=1)]: Done  49 tasks      | elapsed:    0.1s
[Parallel(n_jobs=1)]: Done 199 tasks      | elapsed:    1.0s


0.6620718166784787
OrderedDict([('max_depth', 3), ('n_estimators', 469)])


[Parallel(n_jobs=1)]: Done 449 tasks      | elapsed:    2.3s
[Parallel(n_jobs=1)]: Done  49 tasks      | elapsed:    0.0s
[Parallel(n_jobs=1)]: Done 199 tasks      | elapsed:    0.0s
[Parallel(n_jobs=1)]: Done 449 tasks      | elapsed:    0.0s


In [19]:
accuracy = accuracy_score(y_test, y_pred)
print(f"accuracy --> {accuracy}")

accuracy --> 0.6677367576243981


In [20]:
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)
conf_matrix = confusion_matrix(y_test, y_pred)

print(f"precision --> {precision}")
print(f"recall --> {recall}")
print(f"f1 --> {f1}")
print(conf_matrix)


precision --> 0.0
recall --> 0.0
f1 --> 0.0
[[1248    0]
 [ 621    0]]


c:\Users\S_CSIS-PostGrad\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [10]:
y_pred_prob = clf.predict_proba(X_test)[:, 1]  # Get predicted probabilities for class 1
fpr, tpr, thresholds = roc_curve(y_test, y_pred_prob)
roc_auc = roc_auc_score(y_test, y_pred_prob)

print(f"probabilites --> {y_pred_prob}")
print(f"AUC --> {roc_auc}")

probabilites --> [0.46 0.42 0.44 ... 0.39 0.28 0.33]
AUC --> 0.5013103090548743


[Parallel(n_jobs=1)]: Done  49 tasks      | elapsed:    0.0s


In [71]:
print(y_pred_prob.shape)
print(score_test.shape)

probDiffData = {'WinProb' : y_pred_prob, 'ScoreDifference': score_test}
probDiffDf = pd.DataFrame(probDiffData)

print(probDiffDf)

correlation = probDiffDf['ScoreDifference'].corr(probDiffDf['WinProb'])

print(f"Correlation --> {correlation}")

(1869,)
(1869,)
      WinProb  ScoreDifference
360      0.22                3
9344     0.48                2
5648     0.54                2
8118     0.73                4
7556     0.31                3
...       ...              ...
8801     0.34                4
7640     0.88                3
5017     0.47               11
4694     0.46               12
7909     0.39                8

[1869 rows x 2 columns]
Correlation --> 0.05171471605732767


In [72]:
X_train, X_test, y_train, y_test = train_test_split(scaled_features, ScoreDifference, test_size=0.2, random_state=42)

print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

reg = RandomForestRegressor(n_estimators=100, random_state=42, verbose=2)
reg.fit(X_train, y_train)

y_pred = reg.predict(X_test)

(7472, 10)
(1869, 10)
(7472,)
(1869,)
building tree 1 of 100
building tree 2 of 100
building tree 3 of 100
building tree 4 of 100
building tree 5 of 100
building tree 6 of 100
building tree 7 of 100
building tree 8 of 100
building tree 9 of 100
building tree 10 of 100
building tree 11 of 100
building tree 12 of 100
building tree 13 of 100
building tree 14 of 100
building tree 15 of 100
building tree 16 of 100
building tree 17 of 100
building tree 18 of 100
building tree 19 of 100
building tree 20 of 100
building tree 21 of 100
building tree 22 of 100
building tree 23 of 100
building tree 24 of 100
building tree 25 of 100
building tree 26 of 100
building tree 27 of 100
building tree 28 of 100
building tree 29 of 100
building tree 30 of 100
building tree 31 of 100
building tree 32 of 100
building tree 33 of 100
building tree 34 of 100
building tree 35 of 100
building tree 36 of 100
building tree 37 of 100
building tree 38 of 100
building tree 39 of 100
building tree 40 of 100


[Parallel(n_jobs=1)]: Done  40 tasks      | elapsed:    5.7s


building tree 41 of 100
building tree 42 of 100
building tree 43 of 100
building tree 44 of 100
building tree 45 of 100
building tree 46 of 100
building tree 47 of 100
building tree 48 of 100
building tree 49 of 100
building tree 50 of 100
building tree 51 of 100
building tree 52 of 100
building tree 53 of 100
building tree 54 of 100
building tree 55 of 100
building tree 56 of 100
building tree 57 of 100
building tree 58 of 100
building tree 59 of 100
building tree 60 of 100
building tree 61 of 100
building tree 62 of 100
building tree 63 of 100
building tree 64 of 100
building tree 65 of 100
building tree 66 of 100
building tree 67 of 100
building tree 68 of 100
building tree 69 of 100
building tree 70 of 100
building tree 71 of 100
building tree 72 of 100
building tree 73 of 100
building tree 74 of 100
building tree 75 of 100
building tree 76 of 100
building tree 77 of 100
building tree 78 of 100
building tree 79 of 100
building tree 80 of 100
building tree 81 of 100
building tree 82

[Parallel(n_jobs=1)]: Done  40 tasks      | elapsed:    0.0s


In [73]:
from sklearn.metrics import mean_squared_error

mse = mean_squared_error(y_test, y_pred)
print(f"Mean squared error --> {mse}")

Mean squared error --> 10.464097752808987
